In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
import math
 
# 定义一个简单的Transformer模型


In [18]:
df = pd.read_csv('DynamicNetwork.csv',header = None).values
df_tensor = torch.tensor(df,dtype = torch.float32)

In [19]:
df[0]

array([1.    , 0.0431, 0.8815, 0.8103, 1.    , 0.0431, 0.8815, 0.8103,
       0.4223, 1.    , 0.6499, 0.862 , 0.4223, 1.    , 0.6499, 0.862 ,
       0.2513, 0.0492, 1.    , 0.3717, 0.2513, 0.0492, 1.    , 0.3717,
       0.0077, 0.0126, 0.2778, 1.    , 0.0077, 0.0126, 0.2778, 1.    ,
       1.    , 0.0431, 0.8815, 0.8103, 1.    , 0.0431, 0.8815, 0.8103,
       0.4223, 1.    , 0.6499, 0.862 , 0.4223, 1.    , 0.6499, 0.862 ,
       0.2513, 0.0492, 1.    , 0.3717, 0.2513, 0.0492, 1.    , 0.3717,
       0.0077, 0.0126, 0.2778, 1.    , 0.0077, 0.0126, 0.2778, 1.    ])

In [26]:
def Preprocessing(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if data[i][j] == 0:
                data[i][j] = 1
    return np.abs(data).reshape(data.shape[0],8,8)

In [27]:
df = Preprocessing(df)

In [28]:
def entropy(X):
    E = []
    for i in range(X.shape[0]):
        P = []
        for j in range(X.shape[1]):
            if i !=j:
                e = -X[i][j]*np.log(X[i][j])
                P.append(e)
        P = np.array(P)
        E.append(np.sum(P))
    return np.array(E)

In [29]:
def graphentropy(X):
    E = []
    for i in range(X.shape[0]):
        e = entropy(X[i])
        E.append(np.sum(e))
    return np.array(E)

In [30]:
def distance(x,y):
    distance = np.mean(np.power((x - y),2))
    return distance

In [31]:
def getMatrix(data,E):
    Matrix = []
    for i in range(E.shape[0]):
        dis = []
        for j in range(E.shape[0]):
            dis.append(distance(E[i],E[j]))
        dis = np.array(dis)
        index = np.argsort(dis)[1]
        Matrix.append(data[index])
    return np.array(Matrix)

In [33]:
E = graphentropy(df)

In [36]:
MatrixX = getMatrix(df,E)

In [39]:
df_tensor     = torch.tensor(df, dtype = torch.float32)
Matrix_tensor = torch.tensor(MatrixX, dtype = torch.float32) 

In [41]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

In [42]:
class SelfAttention(nn.Module):
    def __init__(self, d_model, n_head):
        super(SelfAttention, self).__init__()
        self.d_model = d_model
        self.n_head = n_head
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.out = nn.Linear(d_model, d_model)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        Q = self.q_linear(query).view(batch_size, -1, self.n_head, self.d_model // self.n_head).transpose(1, 2)  # (batch_size, n_head, seq_len, d_model/n_head)
        K = self.k_linear(key).view(batch_size, -1, self.n_head, self.d_model // self.n_head).transpose(1, 2)  # (batch_size, n_head, seq_len, d_model/n_head)
        V = self.v_linear(value).view(batch_size, -1, self.n_head, self.d_model // self.n_head).transpose(1, 2)  # (batch_size, n_head, seq_len, d_model/n_head)

        score = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_model / self.n_head)
        if mask is not None:
            score = score.masked_fill(mask == 0, -1e9)
        score = F.softmax(score, dim=-1)
        output = torch.matmul(score, V)
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        return self.out(output)

In [43]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, n_head, dim_feedforward=2048, dropout=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = SelfAttention(d_model, n_head)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, mask=None):
        src2 = self.self_attn(src, src, src, mask)
        src = src + self.dropout(src2)
        src = self.norm1(src)
        src2 = self.linear2(F.relu(self.linear1(src)))
        src = src + self.dropout(src2)
        src = self.norm2(src)
        return src

In [44]:
class TransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layers,d_model):
        super(TransformerEncoder, self).__init__()
        self.layers = nn.ModuleList([encoder_layer for _ in range(num_layers)])
        self.pe = PositionalEncoding(d_model)

    def forward(self, src, mask=None):
        src = self.pe(src)
        for layer in self.layers:
            src = layer(src, mask)
        return src

In [45]:
d_model = 8
n_head = 1
encoder_layer = TransformerEncoderLayer(d_model, n_head)
encoder = TransformerEncoder(encoder_layer, num_layers=6, d_model = d_model)
src = torch.rand(10, 32, d_model)
output = encoder(df_tensor)

In [46]:
output

tensor([[[ 0.2084,  0.5347, -0.5083,  ..., -1.2946, -0.1195, -0.8557],
         [ 0.5662,  0.5660, -0.6121,  ..., -0.8518, -0.2084, -0.9177],
         [ 0.7936, -0.7140, -0.4073,  ..., -1.2953, -0.2157, -0.7398],
         ...,
         [ 0.0549,  0.5111, -0.7392,  ..., -0.9332, -0.0617, -0.9952],
         [-0.4467,  0.2132,  0.0611,  ..., -1.0955, -0.2136, -1.2274],
         [ 0.4209,  0.2480, -0.4988,  ..., -0.9579, -0.5270, -0.8982]],

        [[ 0.2535,  0.5675, -0.7622,  ..., -1.3451, -0.4194, -0.5255],
         [ 0.7244,  0.2664, -0.6460,  ..., -0.8945, -0.1952, -0.8838],
         [ 0.5979, -0.6546, -0.3460,  ..., -1.3370, -0.3843, -0.6283],
         ...,
         [-0.1214,  0.6293, -0.5810,  ..., -0.8245, -0.4508, -0.9992],
         [-0.3745,  0.3311,  0.1021,  ..., -1.4268, -0.0949, -1.1154],
         [ 0.6697,  0.3268, -0.7203,  ..., -0.9297, -0.7516, -0.8998]],

        [[ 0.2099,  0.8953, -0.9504,  ..., -1.1256, -0.1553, -0.8568],
         [ 0.5331,  0.8903, -0.7159,  ..., -0

In [53]:
optimizer = torch.optim.Adam(encoder.parameters(), lr = 0.01)
loss_func = nn.MSELoss()

In [54]:
num_epochs = 100
d_model = 8
n_head = 1
for epoch in range(num_epochs):
    output1 = encoder(df_tensor)
    output2 = encoder(Matrix_tensor)
    loss = loss_func(output1,output2)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss)

tensor(0.0001, grad_fn=<MseLossBackward0>)
tensor(0.0493, grad_fn=<MseLossBackward0>)
tensor(0.0034, grad_fn=<MseLossBackward0>)
tensor(0.0047, grad_fn=<MseLossBackward0>)
tensor(0.0062, grad_fn=<MseLossBackward0>)
tensor(0.0063, grad_fn=<MseLossBackward0>)
tensor(0.0067, grad_fn=<MseLossBackward0>)
tensor(0.0065, grad_fn=<MseLossBackward0>)
tensor(0.0060, grad_fn=<MseLossBackward0>)
tensor(0.0064, grad_fn=<MseLossBackward0>)
tensor(0.0057, grad_fn=<MseLossBackward0>)
tensor(0.0057, grad_fn=<MseLossBackward0>)
tensor(0.0049, grad_fn=<MseLossBackward0>)
tensor(0.0047, grad_fn=<MseLossBackward0>)
tensor(0.0039, grad_fn=<MseLossBackward0>)
tensor(0.0041, grad_fn=<MseLossBackward0>)
tensor(0.0034, grad_fn=<MseLossBackward0>)
tensor(0.0032, grad_fn=<MseLossBackward0>)
tensor(0.0027, grad_fn=<MseLossBackward0>)
tensor(0.0024, grad_fn=<MseLossBackward0>)
tensor(0.0022, grad_fn=<MseLossBackward0>)
tensor(0.0018, grad_fn=<MseLossBackward0>)
tensor(0.0018, grad_fn=<MseLossBackward0>)
tensor(0.00

In [56]:
output1.shape

torch.Size([126, 8, 8])

In [57]:
np.savetxt('Embedding_Netwokr.csv',output1.data.numpy().reshape(126,-1),delimiter=',')